<a href="https://colab.research.google.com/github/JSJeong-me/GPT-Finetuning/blob/main/vLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. 필수 라이브러리 설치
!pip install -q "vllm[triton]" accelerate

# 2. vLLM 서버 실행 (background에서 실행되도록 --host 0.0.0.0)
import os
from subprocess import Popen

model_name = "google/gemma-1.1-2b-it"  # gemma-3-1b-it은 공개되지 않았을 수 있음
port = 8000

# 백그라운드로 vLLM API 서버 실행
server_command = f"python3 -m vllm.entrypoints.openai.api_server --model {model_name} --port {port} --host 0.0.0.0"
process = Popen(server_command.split(), stdout=open("vllm_server.log", "w"))

print(f"vLLM 서버가 포트 {port}에서 실행 중입니다...")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 80.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 130.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7

In [ ]:
!pip install -q pyngrok

In [6]:
from pyngrok import ngrok
from google.colab import userdata
import os

# Set your ngrok authtoken
NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

# ngrok 터널 열기
public_url = ngrok.connect(port)
print(f"🔗 Public URL: {public_url}/v1/chat/completions")

🔗 Public URL: NgrokTunnel: "https://41af-35-198-241-233.ngrok-free.app" -> "http://localhost:8000"/v1/chat/completions


In [9]:
import requests

# Replace with your actual public URL
public_url = "https://41af-35-198-241-233.ngrok-free.app" # Use the URL from the previous cell's output

In [10]:
# Example request to the chat completions endpoint
url = f"{public_url}/v1/chat/completions"
headers = {
    "Content-Type": "application/json"
}
data = {
    "model": "google/gemma-1.1-2b-it",
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is the capital of France?"}
    ]
}

In [ ]:
try:
    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status() # Raise an exception for bad status codes
    print("Response from vLLM API:")
    print(response.json())
except requests.exceptions.RequestException as e:
    print(f"Error making request: {e}")